In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
from IPython.display import display

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras import models
from tensorflow.keras import callbacks

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupKFold

In [2]:
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
train=pd.read_csv(f"{dataset_dir}train.csv")

npz=np.load("../input/effnet_tuned_output.npz")
features_list=[npz[uid] for uid in tqdm(train["StudyInstanceUID"])]
features=np.array(features_list)

  0%|          | 0/30083 [00:00<?, ?it/s]

In [3]:
def normalize(features,save_scaler=True):
    scaler=MinMaxScaler()
    scaler.fit(features)
    if save_scaler:
        pickle.dump(scaler,open("./models/minmaxscaler_effnet_tuned.pickle","wb"))
    X=scaler.transform(features)
    return X

X=pd.DataFrame(normalize(features,save_scaler=False))

## Dropout
直前の層に対してドロップアウトを適用する, 与えられた確率に従ってノードの一部を無効化する

## Sigmoid
活性化関数の1つ, \[0,1]の範囲を出力する

In [4]:
fold=train.copy()
group_kfold=GroupKFold(n_splits=10)
for n,(train_idx,val_idx) in enumerate(group_kfold.split(train,groups=train["PatientID"])):
    fold.loc[val_idx,"folds"]=n
fold["folds"]=fold["folds"].astype(int)

In [5]:
target_fold=0
train_idx=(fold["folds"]!=target_fold)
val_idx=(fold["folds"]==target_fold)

X_train=X[train_idx]
X_val=X[val_idx]
display(X_train)

,0,1,2,3,4,5,6,7,8,9,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,0.190440,0.385044,0.443073,0.310448,0.346378,0.100379,0.092547,0.436334,0.156383,0.402474,...,0.337346,0.326803,0.372029,0.612123,0.039223,0.261128,0.387774,0.152095,0.375815,0.373878
1,0.206376,0.344567,0.164791,0.288545,0.321263,0.444134,0.422946,0.451133,0.157484,0.204861,...,0.235174,0.411999,0.225011,0.209855,0.008203,0.224002,0.226538,0.651499,0.141680,0.172821
2,0.226800,0.252439,0.242553,0.289306,0.322075,0.223481,0.261045,0.265227,0.253216,0.223972,...,0.229770,0.377200,0.246991,0.349057,0.477504,0.249797,0.207370,0.375446,0.269789,0.228729
3,0.194851,0.191336,0.341818,0.273755,0.151161,0.027027,0.271664,0.462765,0.286703,0.270541,...,0.292065,0.227323,0.311901,0.415648,0.327432,0.314091,0.281705,0.519249,0.249901,0.312677
4,0.287443,0.426359,0.383683,0.361628,0.238865,0.493671,0.223484,0.452848,0.250816,0.364319,...,0.369150,0.213200,0.345335,0.267122,0.239928,0.305450,0.357422,0.368382,0.334064,0.341459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,0.305132,0.244647,0.263042,0.293138,0.289404,0.203950,0.146409,0.364405,0.221869,0.226957,...,0.244204,0.408309,0.266780,0.296242,0.007972,0.205312,0.218823,0.213217,0.285859,0.241543
30079,0.344102,0.206995,0.246667,0.204177,0.110451,0.176185,0.236174,0.367238,0.195823,0.212698,...,0.255028,0.166039,0.246962,0.251577,0.382749,0.228371,0.221398,0.462461,0.208752,0.215964
30080,0.204613,0.183264,0.221124,0.176001,0.288095,0.234859,0.130213,0.395777,0.184253,0.172003,...,0.292246,0.436881,0.179868,0.262722,0.009253,0.232791,0.212802,0.258772,0.149610,0.217615
30081,0.153266,0.271216,0.264500,0.270118,0.553315,0.195615,0.260337,0.370405,0.289186,0.236477,...,0.289630,0.553747,0.251055,0.444548,0.398964,0.270383,0.212072,0.436539,0.241500,0.287819


In [6]:
input_shape=(2560,)
encoding_dim=100

model=keras.Sequential([
    Dense(encoding_dim*18,activation="relu",input_shape=input_shape),
    Dense(encoding_dim,activation="relu"),
    Dense(encoding_dim*18,activation="relu"),
    Dropout(0.1),
    Dense(input_shape[0],activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1800)              4609800   
_________________________________________________________________
dense_1 (Dense)              (None, 100)               180100    
_________________________________________________________________
dense_2 (Dense)              (None, 1800)              181800    
_________________________________________________________________
dropout (Dropout)            (None, 1800)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2560)              4610560   
Total params: 9,582,260
Trainable params: 9,582,260
Non-trainable params: 0
_________________________________________________________________


In [7]:
adam=keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=adam,loss="mse")

fit_callbacks=[
    callbacks.EarlyStopping(monitor="val_loss",patience=20,mode="min"),
    callbacks.ReduceLROnPlateau(monitor="val_loss",patience=30,min_lr=1e-7,mode="min",factor=0.5,verbose=1),
    callbacks.ModelCheckpoint("./models/autoencoder_tuned/ckpt",monitor="val_loss",mode="min",
        save_weights_only=True,save_best_only=True)
]

model.fit(X_train,X_train,epochs=200,shuffle=True,validation_data=(X_val,X_val),callbacks=fit_callbacks)

model.save("./models/autoencoder_effnet_tuned")

score=model.evaluate(X_val,X_val,verbose=0)
print(f"\nscore: {score}")

Epoch 1/400
218/847 [======>.......................] - ETA: 58s - loss: 0.0155

KeyboardInterrupt: 

## score
lr e-3: 0.0138  
lr e-4, layers \*2: 0.013181226328015327  
lr e-3, layers \*4 \*2 : 0.014  
lr e-4, layers \*4: 0.013148400001227856  
lr e-4, layers \*4, normalize: 0.0025459351018071175    
lr e-4, layers \*8, normalize: 0.0024564287159591913  
lr 1e-4, layers \*16, norm: 0.0023377728648483753  
lr 1e-4, layers \*16 \*4, norm: 0.002483354415744543  
lr 1e-3, layers \*16 \*4, norm: 0.0037389046046882868  
lr 1e-3, layers \*16 \*2, norm: 0.0035295113921165466  
lr 1e-4, layers \*16 \*2, norm: 0.0023819475900381804  
lr 1e-4, layers \*18, norm: **0.0021908902563154697**  
lr 1e-4, layers \*20, norm: 0.002232010243460536  
lr 1e-4, layers none, norm: 0.0029496345669031143  
lr 1e-4, layers \*18 dropout, norm:  
lr 1e-4, layers \*18, dropout 0.2, norm: 0.0023323060013353825  
lr 1e-4, layers \*18, dropout 0.1, norm: 0.002210501581430435  
lr reducer, layers dropuout norm same: 0.0022298386320471764  
lr reducer ,epochs 500, others same: 0.0021655228920280933  
groupkfold, others same: 0.0022280211560428143  
groupkfold, dropout 0.05, others same: 0.002246299758553505  
groupkfold, batchnorm, without dropout: 0.003952326253056526  
groupkfold, without batchnorm dropout: 0.0022722359281033278  
groupkfold splits10, dropout 0.1, no batchnorm, others same: **0.002182652475312352**